# 行列の便利な公式

## 概要
本記事は 「Matrix Cookbook」 の内容を参考に、行列計算の有用な公式を整理する。  
リンクは以下の通り。

- https://www.math.uwaterloo.ca/~hwolkowi/matrixcookbook.pdf

個人的に重要だと思われる公式をまとめる。
行列計算に関する公式がトピック毎にまとめられており、とても便利である。
行列分解や微分計算などのポイントを簡潔な文と短いコード例で紹介する。Pythonによる実装サンプルも提示する。  

## ソースコード

### github
- jupyter notebook形式のファイルは[こちら](https://github.com/hiroshi0530/wa-src/blob/master/rec/linalg/07/07_nb.ipynb)

### google colaboratory
- google colaboratory で実行する場合は[こちら](https://colab.research.google.com/github/hiroshi0530/wa-src/blob/master/rec/linalg/07/07_nb.ipynb)


## 実行環境
OSはmacOSである。LinuxやUnixのコマンドとはオプションが異なることに注意。

In [3]:
!sw_vers

ProductName:		macOS
ProductVersion:		13.5.1
BuildVersion:		22G90


In [4]:
!python -V

Python 3.9.17


pandasのテーブルを見やすいようにHTMLのテーブルにCSSの設定を行う。

In [5]:
from IPython.core.display import HTML

style = """
<style>
    .dataframe thead tr:only-child th {
        text-align: right;
    }

    .dataframe thead th {
        text-align: left;
        padding: 5px;
    }

    .dataframe tbody tr th {
        vertical-align: top;
        padding: 5px;
    }

    .dataframe tbody tr:hover {
        background-color: #ffff99;
    }

    .dataframe {
        background-color: white;
        color: black;
        font-size: 16px;
    }

</style>
"""
HTML(style)

基本的なライブラリをインポートし watermark を利用してそのバージョンを確認する。
ついでに乱数のseedの設定を行う。

In [6]:
%matplotlib inline
%config InlineBackend.figure_format = 'svg'

In [7]:
import random

import scipy
import numpy as np

import matplotlib
import matplotlib.pyplot as plt

seed = 123
random_state = 123

random.seed(seed)
np.random.seed(seed)


from watermark import watermark

print(watermark(python=True, watermark=True, iversions=True, globals_=globals()))

Python implementation: CPython
Python version       : 3.9.17
IPython version      : 8.17.2

matplotlib: 3.8.1
numpy     : 1.25.2
scipy     : 1.11.2

Watermark: 2.4.3



## 行列計算の基本概念

行列計算は機械学習の核である。  

大規模モデルや深層学習では、行列演算が反復的にあらわれる。  

入力ベクトルを $\mathbf{x}$ とし、線形変換 $\mathbf{A}$ を考える。  

$\mathbf{y} = \mathbf{A}\mathbf{x}$ のような単純な変換が多用される。  

ここで、$\mathbf{A}$ は $ \mathbb{R}^{m \times n} \to \mathbb{R}^{m \times n} $ の写像とみなせる。  

要素ごとの加算やスカラー倍、転置、逆行列、固有値問題などが基本操作である。  
行列微分、行列分解、テンソル計算なども重要である。  

大規模な行列計算では、数値的安定性や計算コストが問題となる。実装時にはNumPyやSciPyなどのPythonライブラリが有効である。  

次節より、Matrix Cookbookを参照した有用な公式を列挙する。

### summary

$$
\begin{aligned}
(\mathbf{A B})^{-1} & =\mathbf{B}^{-1} \mathbf{A}^{-1} \\
(\mathbf{A B C} \ldots)^{-1} & =\ldots \mathbf{C}^{-1} \mathbf{B}^{-1} \mathbf{A}^{-1} \\
\left(\mathbf{A}^T\right)^{-1} & =\left(\mathbf{A}^{-1}\right)^T \\
(\mathbf{A}+\mathbf{B})^T & =\mathbf{A}^T+\mathbf{B}^T \\
(\mathbf{A B})^T & =\mathbf{B}^T \mathbf{A}^T \\
(\mathbf{A B C} \ldots)^T & =\ldots \mathbf{C}^T \mathbf{B}^T \mathbf{A}^T \\
\left(\mathbf{A}^H\right)^{-1} & =\left(\mathbf{A}^{-1}\right)^H \\
(\mathbf{A}+\mathbf{B})^H & =\mathbf{A}^H+\mathbf{B}^H \\
(\mathbf{A B})^H & =\mathbf{B}^H \mathbf{A}^H \\
(\mathbf{A B C} \ldots)^H & =\ldots \mathbf{C}^H \mathbf{B}^H \mathbf{A}^H
\end{aligned}
$$

---

## トレースと基本的な性質

トレースは行列 $\mathbf{A}$ の対角要素和である。  

$ \text{tr}(\mathbf{A}) = \sum_i A_{ii} $ で定義される。  

トレースには以下の有名な性質がある。  

$ \text{tr}(\mathbf{A}+\mathbf{B}) = \text{tr}(\mathbf{A}) + \text{tr}(\mathbf{B}) $  

$ \text{tr}(c\mathbf{A}) = c \times \text{tr}(\mathbf{A}) $ ここで $ c $ はスカラーである。  

$ \text{tr}(\mathbf{A}\mathbf{B}) = \text{tr}(\mathbf{B}\mathbf{A}) $ といった循環性がある。  

これにより、勾配計算でトレースを用いると式を単純化できる。  

例えば、二次形式 $ \mathbf{x}^\top \mathbf{A}\mathbf{x} $ の微分で便利である。  

行列の期待値演算や行列正定値性の判定にも有用である。

### summary

$$
\begin{aligned}
\operatorname{Tr}(\mathbf{A}) & =\sum_i A_{i i} \\
\operatorname{Tr}(\mathbf{A}) & =\sum_i \lambda_i, \quad \lambda_i=\operatorname{eig}(\mathbf{A}) \\
\operatorname{Tr}(\mathbf{A}) & =\operatorname{Tr}\left(\mathbf{A}^T\right) \\
\operatorname{Tr}(\mathbf{A B}) & =\operatorname{Tr}(\mathbf{B} \mathbf{A}) \\
\operatorname{Tr}(\mathbf{A}+\mathbf{B}) & =\operatorname{Tr}(\mathbf{A})+\operatorname{Tr}(\mathbf{B}) \\
\operatorname{Tr}(\mathbf{A B C}) & =\operatorname{Tr}(\mathbf{B C A})=\operatorname{Tr}(\mathbf{C A B}) \\
\mathbf{a}^T \mathbf{a} & =\operatorname{Tr}\left(\mathbf{a a}^T\right)
\end{aligned}
$$

---






$$
\begin{aligned}
e^{\mathbf{A}} & \equiv \sum_{n=0}^{\infty} \frac{1}{n!} \mathbf{A}^n=\mathbf{I}+\mathbf{A}+\frac{1}{2} \mathbf{A}^2+\ldots \\
e^{-\mathbf{A}} & \equiv \sum_{n=0}^{\infty} \frac{1}{n!}(-1)^n \mathbf{A}^n=\mathbf{I}-\mathbf{A}+\frac{1}{2} \mathbf{A}^2-\ldots \\
e^{t \mathbf{A}} & \equiv \sum_{n=0}^{\infty} \frac{1}{n!}(t \mathbf{A})^n=\mathbf{I}+t \mathbf{A}+\frac{1}{2} t^2 \mathbf{A}^2+\ldots \\
\ln (\mathbf{I}+\mathbf{A}) & \equiv \sum_{n=1}^{\infty} \frac{(-1)^{n-1}}{n} \mathbf{A}^n=\mathbf{A}-\frac{1}{2} \mathbf{A}^2+\frac{1}{3} \mathbf{A}^3-\ldots
\end{aligned}
$$


$$
\begin{aligned}
e^{\mathbf{A}} e^{\mathbf{B}} & =e^{\mathbf{A}+\mathbf{B}} \quad \text { if } \quad \mathbf{A B}=\mathbf{B} \mathbf{A} \\
\left(e^{\mathbf{A}}\right)^{-1} & =e^{-\mathbf{A}} \\
\frac{d}{d t} e^{t \mathbf{A}} & =\mathbf{A} e^{t \mathbf{A}}=e^{t \mathbf{A}} \mathbf{A}, \quad t \in \mathbb{R} \\
\frac{d}{d t} \operatorname{Tr}\left(e^{t \mathbf{A}}\right) & =\operatorname{Tr}\left(\mathbf{A} e^{t \mathbf{A}}\right) \\
\operatorname{det}\left(e^{\mathbf{A}}\right) & =e^{\operatorname{Tr}(\mathbf{A})}
\end{aligned}
$$

$$
\begin{aligned}
&\text { 10.1.6 Trigonometric Functions }\\
&\begin{aligned}
& \sin (\mathbf{A}) \equiv \sum_{n=0}^{\infty} \frac{(-1)^n \mathbf{A}^{2 n+1}}{(2 n+1)!}=\mathbf{A}-\frac{1}{3!} \mathbf{A}^3+\frac{1}{5!} \mathbf{A}^5-\ldots \\
& \cos (\mathbf{A}) \equiv \sum_{n=0}^{\infty} \frac{(-1)^n \mathbf{A}^{2 n}}{(2 n)!}=\mathbf{I}-\frac{1}{2!} \mathbf{A}^2+\frac{1}{4!} \mathbf{A}^4-\ldots
\end{aligned}
\end{aligned}
$$



## フロベニウスノルムとベクトル化

行列のノルムでよく使われるものの一つがフロベニウスノルムである。  

$ \| \mathbf{A} \|\_F = \sqrt{\sum_{i,j} A_{ij}^2 } $ で定義される。  

フロベニウスノルムはトレースで表せる。  

$ \| \mathbf{A} \|_F^2 = \text{tr}(\mathbf{A}^\top \mathbf{A}) $ となる。  

ベクトル化は行列を縦に並べて長いベクトルにする操作である。  

$\text{vec}(\mathbf{A})$ は行列 $\mathbf{A}$ を一列のベクトルにする関数である。  

行列積において、$\text{vec}(\mathbf{ABC}) = (\mathbf{C}^\top \otimes \mathbf{A}) \text{vec}(\mathbf{B})$ の関係が有名である。  

ここで $\otimes$ はKronecker積である。  

Kronecker積は行列をブロック化して拡張する操作である。  

これらの公式は行列微分や最適化問題を整理する上で有用である。

---

## 行列微分の基本公式

行列微分は機械学習の勾配計算で本質的な役割を担う。  
例えば、重み行列 $\mathbf{W}$ に対する損失関数 $L(\mathbf{W})$ の勾配を求める場合がある。  
行列微分の代表例として、$ \frac{\partial \text{tr}(\mathbf{W}\mathbf{W}^\top)}{\partial \mathbf{W}} = 2\mathbf{W} $ がある。  
また、$ \frac{\partial \mathbf{x}^\top \mathbf{A}\mathbf{x}}{\partial \mathbf{A}} = \mathbf{x}\mathbf{x}^\top $ といえる。  
これらは行列微分の基本だが、Matrix Cookbookにはさらに多くの公式が掲載されている。  
確率分布のパラメータ微分、カーネル行列の微分なども類似の手法で扱える。  
数値最適化や勾配降下法で行列微分を活用すると効率的である。

---

## 固有値・固有ベクトルに関する公式

固有値問題は推薦システムや次元削減で頻繁に現れる。  
$\mathbf{A}\mathbf{v} = \lambda \mathbf{v}$ の関係を考える。  
固有値 $\lambda$ と固有ベクトル $\mathbf{v}$ は対称行列の場合特に扱いやすい。  
固有値分解 $\mathbf{A} = \mathbf{Q}\mathbf{\Lambda}\mathbf{Q}^\top$ を考える。  
ここで $\mathbf{\Lambda}$ は対角行列、$\mathbf{Q}$ は直交行列である。  
微分として、$ \frac{\partial \lambda_i}{\partial A_{jk}} = Q_{j i} Q_{k i} $ のような公式がある。  
固有ベクトルの微分はやや複雑であるが、Matrix Cookbookで詳細が整理されている。  
これらの公式はスペクトルクラスタリングやPCAなどで有用である。  
実際、PCAで固有値固有ベクトル計算は実装の基礎である。

---

## 行列分解(SVD、QR、Cholesky)の微分

SVDは行列分解で最も使用頻度が高い。  
$\mathbf{A} = \mathbf{U}\mathbf{\Sigma}\mathbf{V}^\top$ として特異値分解を行う。  
特異値 $\sigma_i$ に関する微分は勾配計算で重要である。  
特異値分解は低ランク近似や推薦システムの潜在因子モデルで多用される。  
QR分解やCholesky分解も数値線形代数で頻用される。  
Cholesky分解は共分散行列など正定値行列に対して用いる。  
これらの分解結果を用いた勾配計算はモデル学習で必須である。  
Matrix Cookbookはこれら分解の微分公式も記載しているため有用である。

---

## 勾配計算と最適化への応用

行列微分公式は勾配降下法や確率的勾配降下法に直結する。  
例えば、行列パラメータ $\mathbf{W}$ を持つモデルの損失関数 $L(\mathbf{W})$ を最小化する場合を考える。  
勾配 $\frac{\partial L}{\partial \mathbf{W}}$ を求めるために、Matrix Cookbookの公式は便利である。  
適切な微分公式により、実装が簡潔になり、エラーも減少する。  
最適化問題での反復更新は行列勾配に依存する。  
これにより、収束速度や計算資源の有効利用が期待できる。

---

## 確率モデルや統計的推定への応用

ガウス分布のパラメータ推定は行列微分公式で簡略化可能である。  
分布の対数尤度をパラメータ行列で微分する過程は頻出である。  
共分散行列 $\mathbf{\Sigma}$ に対する対数尤度の微分などが典型例である。  
行列形式の正則化項も微分公式で容易に扱える。  
EMアルゴリズムやMAP推定など、大規模モデルで重用される。  
こうしたモデル最適化では、行列計算公式が反復使用される。

---

## ベクトル化とKronecker積の応用例

モデル構築で勾配を一気にベクトル化できると高速化できる。  
Kronecker積はベクトル化と組み合わさると強力である。  
$\text{vec}(\mathbf{A}\mathbf{B}\mathbf{C}) = (\mathbf{C}^\top \otimes \mathbf{A}) \text{vec}(\mathbf{B})$ は有名な公式である。  
この公式は勾配の並列計算や大規模計算で有用である。  
特に、TensorFlowやPyTorchでの内部実装や高速化で応用される。  
推薦システムでは、ユーザ・アイテム行列をブロック構造で扱う際に役立つ。

---

## 推薦システムへの応用

行列分解は協調フィルタリングで推薦システムの中核手法である。  
ユーザ行列 $\mathbf{U}$ とアイテム行列 $\mathbf{V}$ で評価行列を近似する。  
$ \mathbf{R} \approx \mathbf{U}\mathbf{V}^\top $ と表すモデルが典型的である。  
勾配を求めるには行列微分公式が不可欠である。  
$ \frac{\partial}{\partial \mathbf{U}} \|\mathbf{R}-\mathbf{U}\mathbf{V}^\top\|_F^2 = 2(\mathbf{U}\mathbf{V}^\top - \mathbf{R})\mathbf{V} $  
$ \frac{\partial}{\partial \mathbf{V}} \|\mathbf{R}-\mathbf{U}\mathbf{V}^\top\|_F^2 = 2(\mathbf{U}\mathbf{V}^\top - \mathbf{R})^\top\mathbf{U} $  
このような勾配計算は大規模推薦システムで高速学習に役立つ。  
Pythonでの実装例を以下に示す。

```python
import numpy as np

# 行列データ準備
R_mat = np.random.randn(100, 50)
U_mat = np.random.randn(100, 10)
V_mat = np.random.randn(50, 10)

# 勾配計算例
gradU_mat = 2*(U_mat @ V_mat.T - R_mat) @ V_mat
gradV_mat = 2*(U_mat @ V_mat.T - R_mat).T @ U_mat
```

これらの勾配計算により、反復更新可能である。  
実際の推薦システム構築時には正則化やスパース行列を考慮する。  
Matrix Cookbookによる行列公式は、こうした変形にも対応可能である。

---

## 具体的な行列微分計算例 (Python実装)

行列微分の理解には実装例が有効である。  
例えば、$f(\mathbf{X}) = \|\mathbf{X}\mathbf{X}^\top - \mathbf{I}\|_F^2$ を考える。  
ここで $\mathbf{X} \in \mathbb{R}^{m\times n}$ とし、$\mathbf{I}$ は単位行列とする。  
この関数の勾配を求める。  
$ f(\mathbf{X}) = \text{tr}((\mathbf{X}\mathbf{X}^\top - \mathbf{I})(\mathbf{X}\mathbf{X}^\top - \mathbf{I})) $  
$ = \text{tr}(\mathbf{X}\mathbf{X}^\top\mathbf{X}\mathbf{X}^\top - 2\mathbf{X}\mathbf{X}^\top + \mathbf{I}) $  
微分で重要なのは $\mathbf{X}\mathbf{X}^\top\mathbf{X}\mathbf{X}^\top$ の項である。  
微分結果は、$ \frac{\partial f}{\partial \mathbf{X}} = 4\mathbf{X}\mathbf{X}^\top\mathbf{X} - 2\mathbf{X} $ といえる。  
Pythonでの計算例を示す。

```python
import numpy as np

m, n = 20, 10
X_mat = np.random.randn(m, n)
I_mat = np.eye(m)

def f(X_mat):
    diff_mat = X_mat @ X_mat.T - I_mat
    return np.sum(diff_mat**2)

def grad_f(X_mat):
    # 微分結果を直接実装する
    return 4*X_mat @ X_mat.T @ X_mat - 2*X_mat

val = f(X_mat)
grad_mat = grad_f(X_mat)
print("関数値:", val)
print("勾配のノルム:", np.linalg.norm(grad_mat))
```

このような実装で行列公式の正しさや数値的性質を確認できる。  
数値微分との比較により、実装の正確性をチェックできる。  
大規模問題では自動微分が多用されるが、公式の理解は依然として重要である。

---

## 機械学習タスクでの応用例

勾配ブースティングやニューラルネットで、行列勾配公式が内部で使われている。  
ニューラルネットの重みやバイアスは行列で表現される。  
逆伝播では行列勾配を反復的に計算する。  
RNNやLSTMでは時間方向の展開で行列公式が便利である。  
カーネル法ではカーネル行列の微分がパラメトリックな更新に使われる。  
行列公式はこれらの計算を簡潔かつ安全に導く指針となる。

---

## 行列計算を効率化するための工夫

行列計算は計算量が大きいので、効率化が必須である。  
ブロック行列や並列計算の利用は一般的である。  
メモリアクセスを最適化し、キャッシュヒット率を高める工夫がある。  
GPUやTPUでの行列積高速化は深層学習で必須である。  
行列公式は理論面での簡易化のみならず、実装面の最適化にも道を開く。  
NumPy、PyTorch、TensorFlowといったライブラリは内部で行列最適化を行う。

---

## 実用上のメリットとデメリット

メリットは以下である。  
- 行列勾配計算が自動微分より理解しやすい  
- 数式簡略化によりコードが簡潔になる  
- 数値安定性や計算効率改善に役立つ

デメリットは以下である。  
- 複雑な公式を覚える負担がある  
- 特殊な分解や派生公式は管理困難  
- 自動微分ツールが既に存在するため、全て暗記不要

実務でのバランスが重要である。

---

## 参考にできる情報源

Matrix Cookbookは豊富な行列公式を集約した文献である。  
Wikipediaや学術論文も適宜参照可能である。  
論文中で行列微分や行列分解公式が利用される場面は多い。  
数値最適化の教科書も公式利用のヒントを与える。

---

## 結論

この記事では、行列微分や各種行列公式を概観した。  
機械学習やAIの最適化問題、推薦システム、確率モデルで役立つ公式を整理した。  
特にMatrix Cookbookは多くの公式を網羅的に収集しているため有用である。  
行列勾配計算を理解すれば、実装効率やモデル性能を向上できる。  
学習者や研究者は、公式を参照しながら実装を検証できる。  
結論として、行列公式は理論的・実用的双方で強力なツールである。

【参考文献】  
- https://www.math.uwaterloo.ca/~hwolkowi/matrixcookbook.pdf